In [1]:
import pandas as pd
import numpy as np
from os import listdir
from os.path import isfile, join
import json
import plotly.express as px
import ast

In [7]:
''' EXTRACT TIME RANGE '''
mypath = 'CalculateDetectionMetrics/cropped_data_labeled/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
time_range = np.array(sorted([float(x[19:-5]) for x in list(filter(lambda s: 'json' in s, onlyfiles))]))
time_range = time_range[time_range >= 30]
time_range = np.round(time_range, 1)
metrics_time_range = time_range
metrics_time_range

array([ 30. ,  33. ,  36. ,  39. ,  42.1,  45. ,  48. ,  51. ,  54. ,
        57. ,  60. ,  63. ,  66. ,  69. ,  72. ,  75. ,  78. ,  81. ,
        84. ,  87. ,  90. ,  93. ,  96. ,  99. , 102. , 105. , 108. ,
       111. , 114. , 117. , 120. , 123. , 126. , 129. , 132. , 135. ,
       138. , 141. , 144. , 147. , 150. , 153. , 156. , 159. , 162. ,
       165. , 168. , 171. , 174. , 177. , 180. , 183. , 186. , 189. ,
       192. , 195. , 198. , 201. , 204. , 207. , 210.1, 214. , 217. ,
       220. , 223. , 226. , 229. , 232. , 235. , 238. , 241. , 244. ,
       247. , 250. , 253. , 256. , 259. , 262. , 265. , 268. , 271. ,
       274. , 277. , 280. , 283. , 286. , 289. , 292. , 295. , 298. ,
       301. , 304. , 307. , 310. , 313. , 316. , 319. , 322. , 325. ,
       328. ])

In [8]:
full_time_range = np.arange(30.0, 330.01, 0.1)
vortices = []
for t in full_time_range:
    vortices_coordinates = pd.read_csv(f'BestDetectorResults-Method5/detected_vortices/vortices_time={np.round(t, 1)}0.csv', delimiter='\t', index_col='Vortex')
    k = len(vortices_coordinates)
    vortices.append(k)

In [10]:
df_detected = pd.DataFrame(data=vortices, index=full_time_range, columns=['detected'])
df_detected.index.name='time'

# Plot presenting number of detected vortices in time

In [13]:
fig = px.line(df_detected)
for t in metrics_time_range:
    fig.add_vline(x=t, line_dash="dash", line_color="green", line_width=0.5)
fig.update_layout(font=dict(size=20))
fig.update_layout(xaxis={'title': 'time[au]'}, 
                  yaxis={'title': 'No. of detected vortices'})
fig.update(layout_showlegend=False)

# GRID SEARCH FOR BP METHOD

In [63]:
mypath = 'BP_Method-Find_Local_Minima_in_2D_Array_RESULTS/'
onlyfiles = [f for f in listdir(mypath) if isfile(join(mypath, f))]
files = list(filter(lambda s: 'GRID' in s, onlyfiles))

In [64]:
df_results = pd.DataFrame(data=np.zeros((5,1)), index=['TP','FP','FN','PRECISION','RECALL'], columns=['thres=100; dist=1'])
df_results

,thres=100; dist=1
TP,0.0
FP,0.0
FN,0.0
PRECISION,0.0
RECALL,0.0


In [65]:
for filename in files:
    df_metrics = pd.read_csv(join(mypath,filename), header=None, delimiter='\t')
    thres = int(filename[34:-11])
    dist = int(filename[43:-4])
    df_metrics.columns = ['TIME', 'TP', 'FP', 'FN']
    df_metrics.set_index('TIME', inplace=True)
    tp = df_metrics.sum()[0]
    fp = df_metrics.sum()[1]
    fn = df_metrics.sum()[2]
    precision = 1.0*tp/(tp+fp)
    recall = 1.0*tp/(tp+fn)
    df_results[f'thres={thres}; dist={dist}'] = [tp, fp, fn, precision, recall]
df_results = df_results.round(4).T

In [66]:
df_results[df_results['RECALL'] > 0.96][df_results['PRECISION'] > 0.8][df_results['FP'] < 300].sort_values('FN').head(1)

C:\Users\FILIPK~1\AppData\Local\Temp/ipykernel_20464/1237552346.py:1: UserWarning:

Boolean Series key will be reindexed to match DataFrame index.



,TP,FP,FN,PRECISION,RECALL
thres=110; dist=5,2306.0,298.0,47.0,0.8856,0.98


In [68]:
FILE_TO_SAVE = 'GRID_SEARCH_BP_METHOD_FIND_LOCAL_PEAKS_2D_ARRAY.csv'
df_results.to_csv(join(mypath, FILE_TO_SAVE))